In [1]:
# Required Python packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
from scipy.stats import norm
import copy
import os
import plotly.express as px
import plotly
plotly.offline.init_notebook_mode(connected=True)


#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import get_all_monthly_option_expiries, \
                                                     find_last_thurs_date_of_month, \
                                                     prev_workday_if_holiday, find_wkly_expries,\
                                                     date_of_prev_thurs

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data


In [2]:
#Stock Index of Interest
# stock_ident = "BANKNIFTY"
stock_ident = "NIFTY"

#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "ATM"

#Product type to hedge: either "CE" or "PE"
prod_type = "CE"
# prod_type = "PE"


# The scope of this code is to hedge one option 
# and scope will be extended to a portfolio
no_of_assets = 1
cor_mat = [[1]]

#Path to refer data
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path


# Periods of interest will be a dictionary
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}
# periods_of_interest = {2019: [11, 12], 2020: [1, 2, 3]}
# periods_of_interest = {2019:[8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5]}
# periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}
periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020:[1,2,3,4,5,6,7]}
#August - Good Vol smile
# periods_of_interest = {2020: [3]}

#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]

#Simulation Parameters
no_of_paths = 5000

#Number of options
no_opt=1


In [3]:
# Implied Volatility Surface Plot Generator
periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020:[1,2,3,4,5,6,7]}
# periods_of_interest = {2020: [5]}

#Find the monthly strikes of option from mkt data to find the option to be hedged
#Every month, an option is hedged
mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
dict_wkly_expiries_each_month = find_wkly_expries(mthly_expiries_list, holidays_list)

df_mthly_list = []
for each_month in dict_wkly_expiries_each_month.keys():
    #Please note vol smile is same for European Call and Put
    df_daily_iv_data = pd.read_csv(output_data_path + "A1_" + stock_ident + "_" + "Daily_vol_smile_" + prod_type + "_" + each_month + ".csv")
    df_daily_iv_data = df_daily_iv_data[df_daily_iv_data['Date'] == dict_wkly_expiries_each_month[each_month][0].strftime("%d-%b-%Y")]
    df_daily_iv_data = df_daily_iv_data[df_daily_iv_data['Impl_Vol'] > 0.00001]
    df_daily_iv_data = df_daily_iv_data[df_daily_iv_data['Impl_Vol'] <= 1]
    df_daily_iv_data = df_daily_iv_data.sort_values(['T', 'Moneyness'], ascending=[False, True]).reset_index()
    time_array = np.sort(np.unique(np.array(df_daily_iv_data['T'])))   
    df_daily_iv_data['Tenor'] = df_daily_iv_data.apply(lambda x: "1W" \
                                                       if (x['T'] == time_array[0]) else \
                                                       "1M", axis=1)
    
    
    df_mthly_list.append(df_daily_iv_data)
    #Plotting scatter plot with quadratic fit

df_plot = pd.concat(df_mthly_list, axis=0)
colours_plot = px.colors.qualitative.Plotly
colours_plot =colours_plot + ['rgb(166,118,29)', 'rgb(124,124,124)']

# fig = px.scatter(df_plot, x="Moneyness", y="Impl_Vol", facet_col="Tenor", color="Date", trendline="lowess", \
#                 color_discrete_sequence=px.colors.qualitative.Dark24)
# fig = px.scatter(df_plot, x="Moneyness", y="Impl_Vol", facet_col="Tenor", color="Date", trendline="lowess", \
#                  color_discrete_sequence=colours_plot)
fig = px.scatter(df_plot, x="Moneyness", y="Impl_Vol", facet_col="Tenor", color="Date", trendline="lowess", \
                 color_discrete_sequence=colours_plot)

fig.show() 
# fig.savefig(output_data_path + "Fig_Implied_VolSurface.png")
    


In [14]:
# fig.write_image(output_data_path + "P1_Fig_Implied_VolSurface_OctNOv2019.png")

In [5]:
# print(px.colors.qualitative.Plotly)

In [6]:
# fig = px.colors.qualitative.swatches()
# fig.show()

In [7]:
# colours_plot = px.colors.qualitative.G10
# colours_plot + ['rgb(166,118,29)', 'rgb(124,124,124)']

In [8]:
# print(px.colors.qualitative.Dark2)